In [ ]:
import numpy as np
import pandas as pd
import time
def solgenerator(A_o, b, c_o, M):

  m,n = A_o.shape         #find the shape of the A matrix and assign the number of rows and columns to m and n

  x_o = np.ones((n,1))        #assign a value of 1's as this will be our initial solution for all the initial x's

  A_I = np.eye(m)         #create an identity matrix that is m*m. This will be used to make the new A matrix for the artificial variables.

  Coeff = (b - (A_o@x_o))  #As per the big M find the coefficeints of the new A matrix associated with the artifical variables.
  A_art = A_I*Coeff        #Assign the value of the coefficients to the A matrix of the artificial variables by multiplying them with the identity matrix of these new variables.

  A = np.hstack((A_o,A_art)) #Get the new A matrix by horizontally combining the A matrix of the original variables with the A matrix of the artifical variables.
  x_a = np.ones((m,1))           #Set all the artificial variables to have value of 1, as we do for the BIG M method for interior point.
  x = np.vstack((x_o,x_a))   #Get all of our values of x together in one column, including the 1's we assigned to the artificial variables.

  c_art = (np.ones((m, 1))*M) #find the coefficients of the c values in the objective function by multiplying it with an assigned Big M in the function for the

  c = np.vstack((c_o,c_art))  #make a final c row array that has all the values of c associated with the original variables and the artificial variables

  return A, b, c, x

In [ ]:
def interiorpoint(A, b, c, X, e):
  while True:
    Xk = np.eye(X.shape[0])*X    #forming a diagonal matrix with the initial solution we got from the solution generator
    y = np.ones((X.shape[0],1))   #assigning the initial value of y that will be all 1s

    w_1 = A @ Xk
    w_2 = w_1 @ Xk
    w_3 = w_2 @ c
    A_T = A.T
    w_4 = w_2 @ A_T

    w = np.linalg.solve(w_4, w_3) #solving for the dual values using linalg solve
    r = c - A_T @ w                    #solving for the reduced costs

    yXr_1 = y.T @ Xk
    yXr = yXr_1 @ r #solving for the dual gap to check optimality conditions

    if abs(yXr) <= e: # Optimality check
      X_optimal = X
      OF = c.T @ X_optimal
      return X_optimal, OF

    d_y = -Xk @ r #if we are not at optimality, we compute the direction for y

    alpha = 1000000 #Initializing the alpha values

    for d in d_y: #unboundedness checking and generation of step length
      if d < 0: #we compute the step length only for the direction items that have a negative component associated with it.
        alpha_new = -0.5/d
        if alpha_new < alpha:
          alpha = alpha_new
    X = X + alpha * (Xk @ d_y) #next x_point computation

In [ ]:
Interior_generated_data = pd.DataFrame(columns=['Scenarios_no', 'Objective_function_value', 'Time_needed', 'Largest_percentage_change'])

for i_s in range(1, 302, 4):

  number_of_scenarios = i_s                                    #deciding the number of variables
  percentage = 0.0036                                          #deciding the percentage increment
  probabilities = 1/number_of_scenarios

  scenarios_list = np.array([])
  for scene in range(number_of_scenarios):
    scenarios_list = np.append(scenarios_list, scene+1)
  percentage_list = ((scenarios_list - np.median(scenarios_list))*percentage)+1                #getting the percentage list based on the number of scenarios

  #it was noticed that the A matrix followed a certain pattern when the scenarios increased. so it's been devided into 4 parts:
  #the initial constraint part (x1+x2+x3+s1 = 500), The coefficient of land acres which will be multiplied with the percentage, other variables coefficients which stays the same and form a super diagonal matrix, the rest of the slacks coefficients which also form a super diagonal matrix
  A_X_normal = np.array([[1, 1, 1, 1]])
  A_X = np.array([[2.5, 0, 0, 0],
                  [0, 3, 0, 0],
                  [0, 0, -20, 0],
                  [0, 0, 0, 0]])
  A_Main_rest = np.array([[1, 0, -1, 0, 0, 0],
                          [0, 1, 0, -1, 0, 0],
                          [0, 0, 0, 0, 1, 1],
                          [0, 0, 0, 0, 1, 0]])
  A_Slacks_rest = np.array([[-1, 0, 0, 0],
                            [0, -1, 0, 0],
                            [0, 0, 1, 0],
                            [0, 0, 0, 1]])
  b_X = np.array([[500]])                                     #the initial constraint b (x1+x2+x3+s1 = 500), is not repeated
  b_Rest = np.array([[200],[240],[0],[6000]])                 #rest of b, repeated as many scenarios
  c_X = np.array([[150],[230],[260], [0]])                    #land acres c, are not repeated
  c_Rest = np.array([[238],[210],[-170],[-150],[-36],[-10]])                        #other variables c, repeated as many scenarios
  c_Slacks_rest = np.array([[0],[0],[0],[0]])                 #slacks c, repeated as many scenarios

  A_Main_rest_new = np.kron(np.eye(number_of_scenarios), A_Main_rest)               #putting this part of the A matrix into the super diagonal matrix form
  A_Main_rest_new[np.isclose(A_Main_rest_new, 0, atol=1e-10)] = 0                   #removing any small values generated instead of 0
  A_Slacks_rest_new = np.kron(np.eye(number_of_scenarios), A_Slacks_rest)
  A_Slacks_rest_new[np.isclose(A_Slacks_rest_new, 0, atol=9e-10)] = 0
  A_X_new = np.vstack([A_X * perc for perc in percentage_list])                     #multiplying this part with our percentage set
  A_Rest_new = np.hstack((A_X_new,A_Main_rest_new,A_Slacks_rest_new))               #stacking the three parts into on matrix
  num_columns = A_Rest_new.shape[1]
  zero_rows = np.zeros((1, num_columns - A_X_normal.shape[1]))
  A = np.vstack([np.hstack([A_X_normal, zero_rows]), A_Rest_new])                   #adding the initial constraint coefficient and getting our full A matrix

  b_Rest_new = np.vstack([b_Rest for num in range(number_of_scenarios)])            #multiplying the b part with as much constraint as we have then adding the constant part
  b = np.vstack([b_X,b_Rest_new])

  c_Rest_new = np.vstack([c_Rest * probabilities for num in range(number_of_scenarios)]) #multiplying the c part with as much constraint as we have then adding the constant part
  c_Slacks_rest_new = np.vstack([c_Slacks_rest for num in range(number_of_scenarios)])
  c = np.vstack((c_X,c_Rest_new,c_Slacks_rest_new))

  A_o = np.round(A, decimals=8)                                                       #rounding the values to eliminate any problem with recurring or decimal values
  A_o[np.isclose(A_o, 0, atol=1e-8)] = 0
  c_o = np.round(c, decimals=8)
  c_o[np.isclose(c_o, 0, atol=1e-8)] = 0
  M = 1000000                           #Initializing the M and epsilon values to usein our Solution Generator (initial solution generator) and interiorpoint function
  epsilon = 0.01

  start_time = time.time()

  A,b,c,x = solgenerator(A_o,b,c_o,M)
  d, y = interiorpoint(A,b,c,x, epsilon)

  end_time = time.time()
  elapsed_time = end_time - start_time

  Interior_generated_data = Interior_generated_data.append({'Scenarios_no': i_s, 'Objective_function_value': y.item(), 'Time_needed': elapsed_time, 'Largest_percentage_change': ((i_s-1)/2)*percentage}, ignore_index=True)

<ipython-input-28-98b44aa837ef>:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Interior_generated_data = Interior_generated_data.append({'Scenarios_no': i_s, 'Objective_function_value': y.item(), 'Time_needed': elapsed_time, 'Largest_percentage_change': ((i_s-1)/2)*percentage}, ignore_index=True)


In [ ]:
Interior_generated_data.to_excel('output.xlsx', index=False, header=False) #the time to go through the scenarios kept increasing, so they've been done in batches, each batch stored in an excel file, to be latered combined and demonstrated